**Implementing the Rivest-Shamir-Adleman algorithm in Python**

**KEY GENERATION**

Big primes were obtained from https://bigprimes.org. For this demonstration, we will be using relatively smaller primes, but note that in actual use cases, the primes will be much larger, so as to increase the computational complexity of the encryption and decryption process tremendously.

In [1]:
# Big primes
p, q = 947, 839
# n
n = p*q
n

794533

Helper functions...

In [2]:
# GCD using Euclid's algorithm
def gcd(a, b):
    if b == 0: return a
    return gcd(b, a%b)
#====================================
# LCM using the result lcd(a,b)*gcd(a,b)=ab
def lcm(a, b):
    return a*b//gcd(a,b)
    # By default here, the return data type is implicitly converted to float.
    # Hence, integer division is done, as seen above.
    # Note that a*b/gcd(a,b) will always be an integer value anyways.
#====================================
# Getting random positive coprime less than x
"""
Instead of obtaining a random coprime of x,
we can specifically obtain a random prime below x
and check if it divides x.
If not, we can return this prime.

There are many efficient methods to obtain
random primes for large numbers.
However, since our demo involves smaller values,
we can stick to basic functions.
In our case, we will search coprimes in general,
not primes alone.
"""
from random import randint
def getRandomCoprime(x):
    # Random starting integer (between 2 and [x/16])
    i = randint(3, x//16 - 1)
    while i < x:
        if gcd(i, x) == 1: return i
        i += 1
    return x-1 # Last resort (we should never get to this point)
#====================================
# Finding modular multiplicative inverse
"""
Modular multiplicative inverse of 'a' modulo 'c' is given by 'b',
where ab mod c = 1 => (a mod c)(a mod c) mod c = 1

Note that modular multiplicative inverse of x modulo z
always exists if x and z are coprime. Hence, since
we will always 'a' such that it is coprime to 'c'
(since we will pass e and n to get d), we don't put
a terminating condition for the loop.
"""
def getModularMultiplicativeInverse(a, c):
    k = 1
    while True:
        if (k*c + 1) % a == 0: return (k*c + 1) // a
        """
        Here, we search for k*c + 1 such that 'a' divides k*c + 1.
        If 'a' divides k*c + 1, this means there exists 'b' such that
        ab = k*c + 1 => ab mod c = 1
        => 'b' is the modular multiplicate inverse of 'a' modulo 'c'.
        Hence, we return b = [(k*c + 1) / a]
        """
        k += 1

Note that in our case, we need not define Carmichael's totient function fully, since we can obtain the value for our numbers very easily using the **lcm** function. Now, obtaining $e$ and $d$ such that $(m^e)^d \equiv m \pmod n$...

In [3]:
# Carmichael's function value for n
lambda_n = lcm(p-1, q-1)
print("lambda(n):", lambda_n)

# Obtaining e and d
e = getRandomCoprime(lambda_n)
d = getModularMultiplicativeInverse(e, lambda_n)
print("e:", e)
print("d:", d)
print("Encryption key:", (n, e))
print("Decryption key:", (n, d))

lambda(n): 396374
e: 20429
d: 393289
Encryption key: (794533, 20429)
Decryption key: (794533, 393289)


Note that encryption key of a certain entity (meant to be public) is given by $(n, e)$, and the decryption key of the same entity (meant to be private) is given by $(n, d)$.

**PLAIN TEXT PARTITIONS**

The message that we intend to encrypt...

In [4]:
msg = "Hello there!"
msg

'Hello there!'

Now, we will partition the message so that when we convert the message to integers, the size of integers that we have to deal with will be less than $n$ i.e. the product of the two large primes assigned at the start. In our case, we will partition the message into strings of size 1, because that makes each partition's integer form smaller than $n$, as required.

In [5]:
partionSize = 1 # Each partition will be 1 characters long or less
partitions = [msg[i:i+partionSize] for i in range(0, len(msg), partionSize)]

# Obtained partitions
partitions

['H', 'e', 'l', 'l', 'o', ' ', 't', 'h', 'e', 'r', 'e', '!']

Partition sizes would mostly be increased for

- Larger $n$ values
- Larger messages

Larger paritions can have more complex encryption processes per partition, hence are generally much more difficult to decrypt, and hence increasing the security in data communication. Larger $n$ values would make the decryption key even harder to deduce, also increasing the securing in data communication.

**PLAIN TEXT CONVERSION TO REVERSIBLE INTEGERS**

Reversible means we can again obtain the original string from the integer. We must convert the message partitions such that every partition's integer form must be be divisible to neither $p$ nor $q$ (the large primes assigned at the start).

In [6]:
# Function to convert a message partition to an integer
def messageToInteger(s):
    return ord(s)*5
"""
NOTE:
The conversion from a string to integer must be more complex
than this in practical applications. This is for demo only
(and because I didn't have time to learn a better method
of string to integer conversion and reconversion).
Complexity of string to integer conversion and reconversion
will increase with increase in partition size.
"""

# List for containing message partitions converted to integers
M = []

# Iterating through message partitions
for p in partitions:
    # Iterating through characters in partitioned string
    x = messageToInteger(p)
    M.append(x)

# Message partitions converted to integers
M

[360, 505, 540, 540, 555, 160, 580, 520, 505, 570, 505, 165]

**ENCRYPTION (OBTAINING CIPHERTEXT FROM PLAINTEXT)**

In [7]:
# Function to obtain the modulus for high powers
# i.e. obtain a^k mod b, where k is a huge number.
"""
This function is faster by an enormous margin,
compared to doing (a**k)%b. Furthermore, the
value of the 'mod' variable is always limited,
and does not keep increasing with k, making it
both extremely time and space efficient.
"""
def crazyMod(a, k, b):
    mod = 1
    for i in range(0, k):
        mod = (mod*(a%b)) % b
    return mod

In [12]:
C = [crazyMod(m, e, n) for m in M]
print("Ciphertexts for each message partition:")
C

Ciphers for each message partition:


[585335,
 717401,
 658839,
 658839,
 425983,
 392280,
 644175,
 280583,
 717401,
 614041,
 717401,
 581725]

**DECRYPTION (OBTAINING PLAINTEXT FROM CIPHERTEXT)**

In [13]:
D = [crazyMod(c, d, n) for c in C]
print("Decrypted plaintexts obtained from ciphertexts:")
D

Decrypted plaintexts obtained from ciphertexts:


[360, 505, 540, 540, 555, 160, 580, 520, 505, 570, 505, 165]

Comparing decrypted message with original message, for demo purposes only...

In [14]:
# This comparison cannot and need not happen in actual use cases.
D == M

True

Obtaining the actual message (in string format)...

In [15]:
def integerToMessage(i):
    return chr(d // 5)
"""
NOTE:
The reconversion from a integer to string must be more complex
than this in practical applications, because of the complexity
of conversion from string to integer. This is for demo only
(and because I didn't have time to learn a better method of
string to integer conversion and reconversion).
Complexity of string to integer conversion and reconversion
will increase with increase in partition size.
"""

received = []
# Iterating through each integer
for d in D:
    received.append(integerToMessage(d))

# Printing the received message partitions
print(received)
# Obtaining the full message
''.join(received)

['H', 'e', 'l', 'l', 'o', ' ', 't', 'h', 'e', 'r', 'e', '!']


'Hello there!'